# nb 08: End-Effector Commands

**Note:**
This section skips the control of the gripper as this is handled by the (default) WSG 50 gripper driver.

In order to control the behavior of the available tools, a dedicated ROS-interface has been implemented which is imported and initialized as sketched below.

In [1]:
from tqdm.notebook import trange, tqdm
import numpy as np
import rospy
import hrr_cobot_robot
import hrr_common
from std_srvs.srv import SetBool

np.set_printoptions(suppress=True, precision=4)

In [2]:
#hrr_common.utils.set_ros_environment(ros_master = 'hrrcobotLinux54')

In [3]:
rospy.init_node("hrr_cobot_tutorial8")

## Arduino EE-tool control


In this example, the ``tool_controller.launch`` file has been started in the namespace of the cobo via

```bash
roslaunch hrr_ee_tools tool_controller.launch
```

In [4]:
#from hrr_cobot_robot.tool_control import ArduinoToolControlInterface

In [5]:
#arduino_controller = ArduinoToolControlInterface.from_ros("/hrr_cobot/")

## Control Handles for ROS-control via digital OUT / IN

In [6]:
from hrr_cobot_robot.tool_control import HrrCobotToolControlInterface
final = HrrCobotToolControlInterface.from_ros("/hrr_cobot")

## control tools via unified API
comment as needed

In [7]:
synced = False
if synced:
    current_handle = arduino_controller
else:
    current_handle = final

### control vacuum valve

1. activate the `vacuum` gripper control mode by setting the tool attribute to ``vacuum``
2. use the ``vacuum`` property to enable / disable the command

**WARNING**: this will only send the command once, and will lead to disabling the vacuum command once the timeout is reached, so preferably use this in a loopish behavior

**NOTE-UPDATE ``v0.2.1``** the procedure is equal for both interfaces

In [ ]:
current_handle.tool = "vacuum"

In [ ]:
current_handle.vacuum = True

In [ ]:
current_handle.vacuum = False

### Shaft grinder control

1. actvate the ``shaftgrinder`` again via setting the ``tool`` attribute to ``shaftgrinder``
2. set the ``rpm`` value of the tool_controller as needed

**WARNING**: Again, another loop is needed to ensure that the tool-controller is not automatically disabled after a timeout is reached.

**NOTE: This function is not available from digital I/O**

In [2]:
current_handle.tool = "shaftgrinder"

NameError: name 'current_handle' is not defined

In [ ]:
current_handle.rpm = 10000

In [ ]:
r = rospy.Rate(2)
for rpm in tqdm((30e3, 20e3, 10e3, 5e3, 1e3)):
    current_handle.rpm = rpm
    r.sleep()

### Shaft grinder control via DOUT

In [ ]:
final.run_shaftgrinder(2.0, 'slow')

In [ ]:
final.run_shaftgrinder(1.0, 'variable')

In [ ]:
final.run_shaftgrinder(0.2, 'full_speed')

### Screwdriver control

Procedure:

activate ``screwdriver`` control again by setting the ``tool`` to ``screwdriver``


2. use available functions in ``HrrCobotControl``-handle

   ```ipython
   ?cobot.run_screwdriver_program
   ?cobot.screwdriver_stop
   ```
3. use commands from tool-controller interface directly
    
  ```ipython
  ?digital_io_controller.screwdriver_program 
  ?digital_io_controller.screwdriver_start 
  ?digital_io_controller.screwdriver_stop 
  ```
  
For digital ROS-controller, there is alos

```ipython
?final.run_screwdriver
```

In [7]:
current_handle.tool = "screwdriver"

NameError: name 'current_handle' is not defined

In [9]:
for i in trange(1, 9):
    if synced:
        current_handle.screwdriver_program = i
        rospy.sleep(1.0)
        current_handle.screwdriver_start()
        rospy.sleep(1.0)
        if synced:    
            current_handle.screwdriver_stop()
    else:
        current_handle.run_screwdriver(dt=1.0, prog=i, sleep_time=0.5)
        rospy.sleep(2.0)

  0%|          | 0/8 [00:00<?, ?it/s]

## Tool change

In [9]:
final.open_tool_changer()

In [10]:
final.close_tool_changer()

In [ ]:
rospy.set_param("/hrr_cobot/use_tool_changer_encoder", True)
tmp = HrrCobotToolControlInterface.from_ros("/hrr_cobot")

In [ ]:
tmp.open_tool_changer()

In [ ]:
tmp.close_tool_changer()

In [ ]:
from hrr_controllers.tool_changer_controller import GimaticPinController
GP = GimaticPinController.from_ros(cobot_prefix="/hrr_cobot")
GP.activate()

In [ ]:
GP.deactivate()

# Using the cobot handle

Below you can see how to use the ``screwdriver`` directly from the cobot handle

In [5]:
import hrr_cobot_robot
rospy.set_param("/hrr_cobot/tool_name", "nothing")
cobot = hrr_cobot_robot.HrrCobotControl.from_ros("/hrr_cobot")

Exception ignored in: <function TransformListener.__del__ at 0x7fda60815700>
Traceback (most recent call last):
  File "/opt/ros/noetic/lib/python3/dist-packages/tf2_ros/transform_listener.py", line 60, in __del__
    self.unregister()
  File "/opt/ros/noetic/lib/python3/dist-packages/tf2_ros/transform_listener.py", line 66, in unregister
    self.tf_sub.unregister()
AttributeError: 'TransformListener' object has no attribute 'tf_sub'
pybullet build time: May  8 2021 05:48:13


[INFO]some functions are not yet compiled. Expect delays upon first call
Hrr-Cobot state:
FT-data:
current force:	[ 0.4861  1.0474 11.2061] [N]
current torque:	[-0.9686  1.1792  0.0339] [Nm]
current wrench:	[ 0.484   1.0649 11.2235 -0.9698  1.1854  0.0477] [N,Nm]
=>in contact:	True
---
Robot-data:
q:		[   1.931     6.9374 -109.7847    5.4566   55.3662   50.6192][°]
q_dot:		[0. 0. 0. 0. 0. 0.][rad/s]
tau:		[0. 0. 0. 0. 0. 0.][Nm]
pos:		[ 0.4726 -0.0222  0.5244][m]
quat:		0.04367 <[-0.4329  0.8979 -0.0672]>
tool-pos:	[ 0.4726 -0.0222  0.5244][m]
tool-rpy:	[128.8491   7.8516 175.2001][°]
robot-status:	ready
---
Hrr-Cobot state:
FT-data:
current force:	[ 0.4944  1.0796 11.3379] [N]
current torque:	[-0.9686  1.1856  0.0594] [Nm]
current wrench:	[ 0.4791  1.0782 11.2672 -0.9707  1.1776  0.052 ] [N,Nm]
=>in contact:	True
---
Robot-data:
q:		[   1.931     6.9374 -109.7847    5.4566   55.3662   50.6192][°]
q_dot:		[0. 0. 0. 0. 0. 0.][rad/s]
tau:		[0. 0. 0. 0. 0. 0.][Nm]
pos:		[ 0.4726 -0.0222  

In [6]:
print(cobot)

In [10]:
cobot.tool

'screwdriver'

In [9]:
cobot.T_B_C_robot

  -0.3938   -0.9192   -0.004733  0.5007    
  -0.9186    0.3934    0.03847   0.04294   
  -0.0335    0.0195   -0.9992    0.1961    
   0         0         0         1         


In [7]:
cobot.change_tool("screwdriver", robot_urdf_prefix="hrr_cobot.")
print(cobot)

In [ ]:
cobot.C_p_CE

In [ ]:
rospy.get_param("/hrr_cobot/tool_name")

In [ ]:
rospy.set_param("/hrr_cobot/tool_name", "wsg_50_dsa")

In [ ]:
cobot.change_tool("screwdriver")

In [8]:
for i in trange(1, 9):
    cobot.run_screwdriver_program(i, run_time=1.0)
    rospy.sleep(2.0)

  0%|          | 0/8 [00:00<?, ?it/s]

In [ ]:
cobot.change_tool("vacuum")

In [ ]:
cobot.tool_controller.vacuum = True

In [ ]:
cobot.tool_controller.vacuum = False

In [ ]:
cobot.tool_controller.tool

## WSG 50 Gripper

In [ ]:
cobot.tool_controller.run_shaftgrinder(1.0, 'slow')

In [ ]:
cobot.tool_controller._sg.mode_id, \
cobot.tool_controller._sg.mode_name, 


In [ ]:
cobot.tool_controller._sg.timeout